In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [10]:
spark = SparkSession.builder.appName("imdb_query").getOrCreate()

24/10/10 22:11:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


70829

In [4]:
import os

imdb_tables = {}

for file in os.listdir("../data"):
    print(file.split(".")[:2])
    imdb_tables["_".join(file.split(".")[:2])] = (
        spark.read.options(
            **{
                "sep": "\t",
                "header": True,
                "compression": "gzip",
                "nullValue": r"\N",
            }
        ).csv(f"../data/{file}")
        # .limit(500)
    )

['title', 'akas']
['title', 'principals']
['title', 'ratings']
['title', 'basics']
['name', 'basics']
['title', 'episode']
['title', 'crew']


In [5]:
import transformation.bronze

In [6]:
bronze_transformation = {
    "name_basics": transformation.bronze.name_basics.transformation,
    "title_akas": transformation.bronze.title_akas.transformation,
    "title_basics": transformation.bronze.title_basics.transformation,
    "title_crew": transformation.bronze.title_crew.transformation,
    "title_episode": transformation.bronze.title_episode.transformation,
    "title_principals": transformation.bronze.title_principals.transformation,
    "title_ratings": transformation.bronze.title_ratings.transformation,
}

cleansed_imdb_tables = {
    _table_name: _sdf.transform(bronze_transformation[_table_name])
    for _table_name, _sdf in imdb_tables.items()
}

---

# Task 1

## Retrieve the top 10 movies with a minimum of 500 votes with the ranking determined by

`(numVotes/averageNumberOfVotes)*averageRating`

In [7]:
# Getting a list of movies
title_movie_sdf = (
    cleansed_imdb_tables["title_basics"]
    .filter(F.col("titleType") == "movie")
    .select("tconst")
)

# Getting the value of the average number of votes across all titles including movies and short
average_number_of_voters = (
    cleansed_imdb_tables["title_ratings"]
    .agg(F.avg("numVotes").alias("averageNumberOfVotes"))
    .head()["averageNumberOfVotes"]
)

print(f"{average_number_of_voters = }")

# Ranking Logics is given from the PDF
ranking_logics = (
    F.col("numVotes") * F.col("averageRating") / F.lit(average_number_of_voters)
)
ranked_title_sdf = (
    cleansed_imdb_tables["title_ratings"]
    .join(
        # Filtering the ratings table to movie only
        title_movie_sdf,
        on="tconst",
        how="inner",
    )
    .select(
        "tconst",
        F.col("numVotes"),
        ranking_logics.alias("ranking"),
    )
)

top_10_movies_with_min_500_votes_sdf = (
    ranked_title_sdf.filter(F.col("numVotes") >= 500)
    .orderBy(F.col("ranking").desc())
    .limit(10)
)

top_10_movies_with_min_500_votes_with_title_sdf = (
    top_10_movies_with_min_500_votes_sdf.join(
        cleansed_imdb_tables["title_basics"].select("tconst", "primaryTitle"),
        on="tconst",
        how="left",
    )
)

top_10_movies_with_min_500_votes_with_title_sdf.show(truncate=False)
top_10_movies_with_min_500_votes_with_title_sdf.cache()

average_number_of_voters = 1030.7113621171118


24/10/10 22:06:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+---------+--------+------------------+---------------------------------------------+
|tconst   |numVotes|ranking           |primaryTitle                                 |
+---------+--------+------------------+---------------------------------------------+
|tt0816692|2169406 |16838.12620863304 |Interstellar                                 |
|tt0167260|2018050 |17621.276593569113|The Lord of the Rings: The Return of the King|
|tt0468569|2928852 |25574.248008536997|The Dark Knight                              |
|tt1375666|2599998 |20180.221897696185|Inception                                    |
|tt0137523|2380156 |18473.88968419705 |Fight Club                                   |
|tt0068646|2055097 |17944.76385901959 |The Godfather                                |
|tt0110912|2264054 |17572.748943794046|Pulp Fiction                                 |
|tt0111161|2948283 |25743.91723546857 |The Shawshank Redemption                     |
|tt0109830|2306230 |17900.10344128106 |Forrest Gump   

DataFrame[tconst: string, numVotes: int, ranking: double, primaryTitle: string]

# Task 2

## For these 10 movies, list the persons who are most often credited

In [8]:
top10_movies_with_credited_person_sdf = (
    top_10_movies_with_min_500_votes_with_title_sdf.select("tconst").join(
        cleansed_imdb_tables["title_principals"].select("tconst", "nconst"),
        on="tconst",
        how="inner",  # Some data is missing in title_principals
    )
)

most_credited_person_for_top10_movies_sdf = (
    top10_movies_with_credited_person_sdf.groupBy("nconst")
    .count()
    .orderBy(F.col("count").desc())
    .limit(1)
)

most_credited_person_for_top10_movies_with_names_sdf = (
    most_credited_person_for_top10_movies_sdf.join(
        cleansed_imdb_tables["name_basics"].select(
            "nconst",
            "primaryName",
        ),
        on="nconst",
        how="left",
    )
)

most_credited_person_for_top10_movies_with_names_sdf.show()

+---------+-----+-----------------+
|   nconst|count|      primaryName|
+---------+-----+-----------------+
|nm0634240|    9|Christopher Nolan|
+---------+-----+-----------------+



## For these 10 movies, list the different titles of the 10 movies

In [9]:
top_10_movies_with_aka_sdf = top_10_movies_with_min_500_votes_with_title_sdf.select(
    "tconst", "primaryTitle"
).join(
    cleansed_imdb_tables["title_akas"].select(
        "tconst", "title", "region", "isOriginalTitle"
    ),
    on="tconst",
    how="left",
)
top_10_movies_with_aka_sdf.groupBy("tconst", "primaryTitle").agg(
    F.collect_set("title")
).show(truncate=False, vertical=True, n=10)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------